# LIBARIRIES

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split,StratifiedKFold,cross_val_score,cross_validate
from sklearn.metrics import classification_report,make_scorer,confusion_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import NearestNeighbors
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier,RandomForestClassifier
from xgboost import XGBClassifier

import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import ListedColormap

from sklearn.preprocessing import MinMaxScaler,StandardScaler

from imblearn.over_sampling import RandomOverSampler,SMOTE,ADASYN,BorderlineSMOTE,SMOTEN,SVMSMOTE,KMeansSMOTE
from imblearn.under_sampling import ClusterCentroids,RandomUnderSampler,NearMiss
from imblearn.combine import SMOTEENN,SMOTETomek

from collections import Counter

from sklearn.cluster import DBSCAN

In [2]:
random_state = 23873

# DATA IMPORT

In [3]:
dt = pd.read_excel("C:\\Users\\PRACHI PANDEY\\Downloads\CA_BC.xlsx").T
dt.columns = dt.iloc[0]
dt = dt.iloc[1:]
dt['Label'] = dt.Label.map({'E':1,'A':0})
dt

Hugo_Symbol,Label,LOC100130426,UBE2Q2P3,UBE2Q2P3,LOC149767,TIMM23,MOXD2,LOC155060,RNU12-2P,SSX9,...,ZXDA,ZXDB,LOC100130182,ZYG11A,ZYG11B,ZYX,FLJ10821,ZZZ3,HUMRTVLH3,AKR1C6P
BA-A1-A0SO,1,0,12.5687,21.4958,383.9288,669.9357,0,143.0169,1.0814,0,...,71.6436,561.2536,374.1691,108.4117,1791.9022,1341.2231,821.0632,897.8436,444.461,0
BA-A2-A04Q,1,0,1.1801,3.8334,185.5958,1706.1319,0,354.8014,0.3857,0,...,37.0228,298.496,885.4609,127.2657,728.4998,4357.1153,896.6448,568.0679,26.2245,0
BA-A2-A04T,1,0,16.6878,23.9972,232.6558,758.0631,0,165.8698,0.3477,0,...,88.6725,739.2854,1018.8646,155.7854,1028.6012,3453.7077,1616.274,655.1334,401.9821,0
BA-A2-A04U,1,0,0,0.5195,411.4041,4388.1024,0,78.9713,0,0,...,17.6646,174.0486,875.4384,130.4065,589.1674,3222.7562,617.7426,1264.0603,85.7254,0
BA-A2-A0D2,1,0,9.8281,9.392,177.6492,1826.2798,0,164.8494,0,0,...,41.0275,402.1438,906.6716,185.548,1189.7986,10575.1247,743.6703,928.1094,20.329,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BA-EW-A1PB,0,0,0,6.7771,109.9378,1960.8476,0,157.8102,0.6454,0,...,47.1171,284.9619,807.7685,140.0605,732.252,5014.4256,791.9552,1232.1455,5.1635,0
BA-EW-A3U0,0,0,7.8621,27.6728,110.0897,755.9254,0,211.9983,0.4038,0,...,57.3405,298.4128,1150.0403,4.8457,758.3482,6454.4395,1530.4258,863.3378,259.6474,0
BA-EW-A6SB,0,0,3.4649,4.6805,211.8421,1575.8145,0,148.4962,0,0,...,41.3534,325.8145,901.6291,266.9173,392.8571,5603.3835,683.584,846.4912,69.5489,0
BA-GI-A2C9,0,0,17.295,23.1124,166.0144,1334.7804,0,284.1877,0.3339,0,...,61.7799,457.5054,623.4764,252.4628,936.3834,5037.5689,646.8526,1224.5784,27.3835,0


In [4]:
dt.drop_duplicates(keep='first',inplace=True)
for x in dt.select_dtypes(include=['object']).columns:
    dt[x] = dt[x].astype('float64')
dt.columns = [str(x) for x in dt.columns]
dt = dt.select_dtypes(include=['float64','int64'])
dt.head(10)

,Label,LOC100130426,UBE2Q2P3,UBE2Q2P3,LOC149767,TIMM23,MOXD2,LOC155060,RNU12-2P,SSX9,...,ZXDA,ZXDB,LOC100130182,ZYG11A,ZYG11B,ZYX,FLJ10821,ZZZ3,HUMRTVLH3,AKR1C6P
BA-A1-A0SO,1,0.0,12.5687,21.4958,383.9288,669.9357,0.0,143.0169,1.0814,0.0,...,71.6436,561.2536,374.1691,108.4117,1791.9022,1341.2231,821.0632,897.8436,444.4610,0.0000
BA-A2-A04Q,1,0.0,1.1801,3.8334,185.5958,1706.1319,0.0,354.8014,0.3857,0.0,...,37.0228,298.4960,885.4609,127.2657,728.4998,4357.1153,896.6448,568.0679,26.2245,0.0000
BA-A2-A04T,1,0.0,16.6878,23.9972,232.6558,758.0631,0.0,165.8698,0.3477,0.0,...,88.6725,739.2854,1018.8646,155.7854,1028.6012,3453.7077,1616.2740,655.1334,401.9821,0.0000
BA-A2-A04U,1,0.0,0.0000,0.5195,411.4041,4388.1024,0.0,78.9713,0.0000,0.0,...,17.6646,174.0486,875.4384,130.4065,589.1674,3222.7562,617.7426,1264.0603,85.7254,0.0000
BA-A2-A0D2,1,0.0,9.8281,9.3920,177.6492,1826.2798,0.0,164.8494,0.0000,0.0,...,41.0275,402.1438,906.6716,185.5480,1189.7986,10575.1247,743.6703,928.1094,20.3290,0.0000
BA-A2-A0SX,1,0.0,12.3539,5.6294,151.5928,840.7193,0.0,156.0694,0.3211,0.0,...,21.8369,335.2601,1333.9756,163.1342,1028.5806,3682.4021,2107.8998,1145.4721,221.9011,0.0000
BA-A2-A0T0,1,0.0,8.6466,4.4593,195.2648,1653.3096,0.0,107.4684,0.0000,0.0,...,26.5395,327.3199,1294.5355,94.6901,1013.7415,3375.0975,706.0805,410.8700,618.9262,0.0000
BA-A2-A0T2,1,0.0,20.3388,5.9652,128.2568,3062.8622,0.0,99.4204,0.8917,0.0,...,58.8498,327.2403,1315.2029,256.3531,999.5542,5042.7998,619.2599,854.2131,13.3749,0.8917
BA-A2-A0YE,1,0.0,23.8942,20.7826,151.7269,721.7997,0.0,209.7592,0.3169,0.0,...,49.4297,422.3701,673.6375,26.2991,806.7174,2441.3815,798.1622,572.5602,6.0203,0.0000
BA-A2-A1G6,1,0.0,7.8319,5.1768,83.8159,775.1835,0.0,213.4757,1.0007,0.0,...,60.7071,580.3869,1397.9320,54.0360,1207.1381,6312.5417,1344.2295,1375.5837,257.8386,0.0000


In [5]:
# Columns to not rename
excluded = dt.columns[~dt.columns.duplicated(keep=False)]

# An incrementer
import itertools
inc = itertools.count().__next__

# A renamer
def ren(name):
    return f"{name}{inc()}" if name not in excluded else name

# Use inside rename()
dt.rename(columns=ren,inplace=True)
dt

,Label,LOC100130426,UBE2Q2P30,UBE2Q2P31,LOC149767,TIMM23,MOXD2,LOC155060,RNU12-2P,SSX9,...,ZXDA,ZXDB,LOC100130182,ZYG11A,ZYG11B,ZYX,FLJ10821,ZZZ3,HUMRTVLH3,AKR1C6P
BA-A1-A0SO,1,0.0,12.5687,21.4958,383.9288,669.9357,0.0,143.0169,1.0814,0.0,...,71.6436,561.2536,374.1691,108.4117,1791.9022,1341.2231,821.0632,897.8436,444.4610,0.0
BA-A2-A04Q,1,0.0,1.1801,3.8334,185.5958,1706.1319,0.0,354.8014,0.3857,0.0,...,37.0228,298.4960,885.4609,127.2657,728.4998,4357.1153,896.6448,568.0679,26.2245,0.0
BA-A2-A04T,1,0.0,16.6878,23.9972,232.6558,758.0631,0.0,165.8698,0.3477,0.0,...,88.6725,739.2854,1018.8646,155.7854,1028.6012,3453.7077,1616.2740,655.1334,401.9821,0.0
BA-A2-A04U,1,0.0,0.0000,0.5195,411.4041,4388.1024,0.0,78.9713,0.0000,0.0,...,17.6646,174.0486,875.4384,130.4065,589.1674,3222.7562,617.7426,1264.0603,85.7254,0.0
BA-A2-A0D2,1,0.0,9.8281,9.3920,177.6492,1826.2798,0.0,164.8494,0.0000,0.0,...,41.0275,402.1438,906.6716,185.5480,1189.7986,10575.1247,743.6703,928.1094,20.3290,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BA-EW-A1PB,0,0.0,0.0000,6.7771,109.9378,1960.8476,0.0,157.8102,0.6454,0.0,...,47.1171,284.9619,807.7685,140.0605,732.2520,5014.4256,791.9552,1232.1455,5.1635,0.0
BA-EW-A3U0,0,0.0,7.8621,27.6728,110.0897,755.9254,0.0,211.9983,0.4038,0.0,...,57.3405,298.4128,1150.0403,4.8457,758.3482,6454.4395,1530.4258,863.3378,259.6474,0.0
BA-EW-A6SB,0,0.0,3.4649,4.6805,211.8421,1575.8145,0.0,148.4962,0.0000,0.0,...,41.3534,325.8145,901.6291,266.9173,392.8571,5603.3835,683.5840,846.4912,69.5489,0.0
BA-GI-A2C9,0,0.0,17.2950,23.1124,166.0144,1334.7804,0.0,284.1877,0.3339,0.0,...,61.7799,457.5054,623.4764,252.4628,936.3834,5037.5689,646.8526,1224.5784,27.3835,0.0


In [6]:
Counter(dt.columns).most_common()

[('CCDC78', 2),
 ('Label', 1),
 ('LOC100130426', 1),
 ('UBE2Q2P30', 1),
 ('UBE2Q2P31', 1),
 ('LOC149767', 1),
 ('TIMM23', 1),
 ('MOXD2', 1),
 ('LOC155060', 1),
 ('RNU12-2P', 1),
 ('SSX9', 1),
 ('LOC317712', 1),
 ('CXORF67', 1),
 ('EFCAB8', 1),
 ('SRP14P1', 1),
 ('LOC391343', 1),
 ('TRIM75P', 1),
 ('SPATA31B1P', 1),
 ('LOC286106', 1),
 ('SDR16C6P', 1),
 ('LOC553137', 1),
 ('KIAA1618', 1),
 ('LOC6458512', 1),
 ('RGPD7', 1),
 ('HSPB1P1', 1),
 ('PPBPL1', 1),
 ('LOC594835', 1),
 ('ANKRD20A20P', 1),
 ('TMPRSS11E2', 1),
 ('LOC100132347', 1),
 ('EFCAB12', 1),
 ('LOC147680', 1),
 ('A1CF', 1),
 ('RBFOX1', 1),
 ('GGACT', 1),
 ('A2ML1', 1),
 ('A2M', 1),
 ('P1', 1),
 ('A4GNT', 1),
 ('NPSR1-AS1', 1),
 ('AAAS', 1),
 ('LOC391856', 1),
 ('AACS', 1),
 ('AADACL2', 1),
 ('LOC649285', 1),
 ('AADACL4', 1),
 ('AADAC', 1),
 ('AADAT', 1),
 ('AAGAB', 1),
 ('DKFZp686K16132', 1),
 ('AAMP', 1),
 ('AANAT', 1),
 ('LOC221410', 1),
 ('AARSD1', 1),
 ('AARS', 1),
 ('AASDHPPT', 1),
 ('AASDH', 1),
 ('AASS', 1),
 ('AATF', 

# TRAIN AND TEST SPLIT DATA

In [7]:
features = dt.copy()
target = features.pop('Label')

# ros = RandomUnderSampler(sampling_strategy = 0.1,random_state = random_state)
# features, target = ros.fit_resample(features, target)
# print(sorted(Counter(target).items()))


x_train, x_test, y_train, y_test = train_test_split(features, target,stratify=target, test_size=0.2)
y_train.shape

(102,)

# SCORERS

#### tss

In [8]:
def tss(y_true,y_pred,**kwargs):
    tn, fp, fn, tp = confusion_matrix(y_true,y_pred).ravel()
    p = tp + fn 
    n = fp + tn
    tss = (tp/p) - (fp/n)
    return tss

In [9]:
# tss([0, 1, 0, 1], [1, 1, 1, 0])
tss_scorer = make_scorer(tss,greater_is_better=True)

#### hss2


In [10]:
def hss2(y_true,y_pred,**kwargs):
    tn, fp, fn, tp = confusion_matrix(y_true,y_pred).ravel()
    p = tp + fn 
    n = fp + tn
    hss2 = (2 * ((tp*tn) - (fn*fp))) / (p*(fn+tn) + n*(tp+fp))
    return hss2

In [11]:
# hss2([0, 1, 0, 1], [1, 1, 1, 0])
hss2_scorer = make_scorer(hss2,greater_is_better=True)

# BASELINE MODELING

In [12]:
# Decision Tree
# Logistic Regression
# KNeighbours
# Support Vector Machine
# Naive Bayes
# AdaBoost
# RandomForest
# ExtraTrees
# GradientBoosting
# Linear Discriminant Analysis
# XGBoost

In [13]:
def baseline_modelling(x_train, y_train,scoring):
    
    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=random_state)
    means = []
    std = []

    classifiers = [LogisticRegression(random_state=random_state),DecisionTreeClassifier(random_state=random_state),
                  SVC(random_state=random_state),NearestNeighbors(),GaussianNB(),AdaBoostClassifier(random_state=random_state),
                    RandomForestClassifier(n_estimators=20,random_state=random_state),XGBClassifier()
                  ]

    for classifier in classifiers:
            scores = cross_val_score(classifier, x_train, y_train, scoring=scoring, cv=cv)
            means.append(scores.mean())
            std.append(scores.std())

    cv_res = pd.DataFrame(data={'Algorithms':['LogisticRegression','DecisionTree','SVM','KNN','Naive Bayes','AdaBoost',
                                              'RandomForest','XGBoost'], 'Mean Score':means, 'std':std})

    sns.barplot(x = 'Mean Score',y = 'Algorithms',data = cv_res, palette = "Set2").set(title='Cross Validation Scores')
    plt.show()

    return cv_res

In [15]:
def modelling_mulitple(x_train, y_train):

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)
    f1_means,f1_std,tss_mean,tss_std,hss2_mean,hss2_std = [],[],[],[],[],[]

    classifiers = [LogisticRegression(random_state=random_state),DecisionTreeClassifier(random_state=random_state),
                  SVC(random_state=random_state),GaussianNB(),AdaBoostClassifier(random_state=random_state),
                    RandomForestClassifier(n_estimators=20,random_state=random_state),XGBClassifier()
                  ]

    scoring = {'f1_macro': 'f1_macro',
               'tss': tss_scorer,
               'hss2': hss2_scorer}

    for classifier in classifiers:
        print(classifier)
        scores = cross_validate(classifier, x_train, y_train, scoring=scoring, cv=cv)
        hybrid_model(x_train,y_train,scoring=scoring)
        f1_means.append(scores['test_f1_macro'].mean())
        f1_std.append(scores['test_f1_macro'].std())
        tss_mean.append(scores['test_tss'].mean())
        tss_std.append(scores['test_tss'].std())
        hss2_mean.append(scores['test_hss2'].mean())
        hss2_std.append(scores['test_hss2'].std())

    cv_res = pd.DataFrame(data={'Algorithms':['LogisticRegression','DecisionTree','SVM','Naive Bayes','AdaBoost',
                                              'RandomForest','XGBoost'], 'F1 Mean Score':f1_means, 'F1 std':f1_std,
                               'TSS Mean Score':tss_mean, 'TSS std':tss_std, 'HSS2 Mean Score':hss2_mean, 'HSS2 std':hss2_std
                               })

    plt.rcParams["figure.figsize"] = [20.00, 6.00]
    plt.rcParams["figure.autolayout"] = True
    f, axes = plt.subplots(1, 3)

    sns.barplot(x = 'F1 Mean Score',y = 'Algorithms',data = cv_res, palette = "Set2",ax=axes[0]).set(title='Cross Validation Scores')
    sns.barplot(x = 'TSS Mean Score',y = 'Algorithms',data = cv_res, palette = "Set2",ax=axes[1]).set(title='Cross Validation Scores')
    sns.barplot(x = 'HSS2 Mean Score',y = 'Algorithms',data = cv_res, palette = "Set2",ax=axes[2]).set(title='Cross Validation Scores')

    plt.show()

    return cv_res

HYBRID MODEL

In [19]:
#HYBRID MODEL

from sklearn.ensemble import AdaBoostClassifier
import xgboost as xgb
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

from sklearn.ensemble import VotingClassifier

f1_means,f1_std,tss_mean,tss_std,hss2_mean,hss2_std = [],[],[],[],[],[]

scoring = {'f1_macro': 'f1_macro',
               'tss': tss_scorer,
               'hss2': hss2_scorer}

# Step 1: Train an AdaBoost model
adaboost_model = AdaBoostClassifier(n_estimators=100)
adaboost_model.fit(x_train, y_train)

# Check for duplicate column names
duplicate_columns = set()
for i in range(x_train.shape[1]):
    col = x_train.columns[i]
    if col in x_train.columns[i + 1:]:
        duplicate_columns.add(col)

# Remove duplicate columns from x_train
x_train = x_train.drop(columns=duplicate_columns)

# Check for duplicate column names
duplicate_columns = set()
for i in range(x_test.shape[1]):
    col = x_test.columns[i]
    if col in x_test.columns[i + 1:]:
        duplicate_columns.add(col)

# Remove duplicate columns from x_train
x_test = x_test.drop(columns=duplicate_columns)

# Step 2: Create a DMatrix for XGBoost
dtrain = xgb.DMatrix(x_train, label=y_train)

# Step 3: Train an XGBoost model
xgb_model = xgb.XGBClassifier(n_estimators=100)
xgb_model.fit(x_train, y_train)

# Step 4: Create a voting classifier
voting_classifier = VotingClassifier(estimators=[('adaboost', adaboost_model), ('xgboost', xgb_model)], voting='hard')
voting_classifier.fit(x_train, y_train)

# Step 5: Predict using the voting classifier
voting_pred = voting_classifier.predict(x_test)

# Step 6: Calculate the F1 score for the hybrid model
f1 = f1_score(y_test, voting_pred)
print("Hybrid Model F1 Score:", f1)

Hybrid Model F1 Score: 0.972972972972973


In [ ]:
# baseline_modelling(x_train, y_train,'f1_macro')
# baseline_modelling(x_train, y_train,tss_scorer)
# baseline_modelling(x_train, y_train,hss2_scorer)

# DATA NORMALIATION

### Min Max Scaler

In [ ]:
x_train_MinMax = MinMaxScaler().fit_transform(x_train)

In [ ]:
modelling_mulitple(x_train_MinMax, y_train)

### Standard Scalar

In [ ]:
x_train_Standard = StandardScaler().fit_transform(x_train)

In [ ]:
modelling_mulitple(x_train_Standard, y_train)

# DATA RE-SAMPLING

## Over Sampling

In [ ]:
# Random Over Sampling
# Random Undersampling
# SMOTE
# SMOTE and Undersmapling
# Random Over Sampling and Undersmapling
# BorderLine
# ADASYN
# SVMSMOTE

#https://imbalanced-learn.org/stable/over_sampling.html

#### Random Over Sampling

In [ ]:
# ros = RandomOverSampler(random_state = random_state)
# x_train_samp, y_train_samp = ros.fit_resample(x_train, y_train)
# print(sorted(Counter(y_train_samp).items()))
# modelling_mulitple(StandardScaler().fit_transform(x_train_samp), y_train_samp)

#### SMOTE

In [ ]:
# ros = SMOTE(random_state = random_state)
# x_train_samp, y_train_samp = ros.fit_resample(x_train, y_train)
# print(sorted(Counter(y_train_samp).items()))
# modelling_mulitple(StandardScaler().fit_transform(x_train_samp), y_train_samp)

#### ADASYN

In [ ]:
# ros = ADASYN(random_state = random_state)
# x_train_samp, y_train_samp = ros.fit_resample(x_train, y_train)
# print(sorted(Counter(y_train_samp).items()))
# modelling_mulitple(StandardScaler().fit_transform(x_train_samp), y_train_samp)

#### BorderlineSMOTE

In [ ]:
# ros = BorderlineSMOTE(random_state = random_state)
# x_train_samp, y_train_samp = ros.fit_resample(x_train, y_train)
# print(sorted(Counter(y_train_samp).items()))
# modelling_mulitple(StandardScaler().fit_transform(x_train_samp), y_train_samp)

#### SVMSMOTE

In [ ]:
# ros = SVMSMOTE(random_state = random_state)
# x_train_samp, y_train_samp = ros.fit_resample(x_train, y_train)
# print(sorted(Counter(y_train_samp).items()))
# modelling_mulitple(StandardScaler().fit_transform(x_train_samp), y_train_samp)

#### SMOTEN

In [ ]:
# ros = SMOTEN(random_state = random_state)
# x_train_samp, y_train_samp = ros.fit_resample(x_train, y_train)
# print(sorted(Counter(y_train_samp).items()))
# modelling_mulitple(StandardScaler().fit_transform(x_train_samp), y_train_samp)

#### KMeansSMOTE

In [ ]:
# ros = KMeansSMOTE(random_state = random_state)
# x_train_samp, y_train_samp = ros.fit_resample(x_train, y_train)
# print(sorted(Counter(y_train_samp).items()))
# modelling_mulitple(x_train_samp, y_train_samp)

## Under Sampling

#### ClusterCentroids

In [ ]:
# ros = ClusterCentroids(random_state = random_state)
# x_train_samp, y_train_samp = ros.fit_resample(x_train, y_train)
# print(sorted(Counter(y_train_samp).items()))
# modelling_mulitple(StandardScaler().fit_transform(x_train_samp), y_train_samp)

#### RandomUnderSampler

In [ ]:
# ros = RandomUnderSampler(random_state = random_state)
# x_train_samp, y_train_samp = ros.fit_resample(x_train, y_train)
# print(sorted(Counter(y_train_samp).items()))
# modelling_mulitple(StandardScaler().fit_transform(x_train_samp), y_train_samp)

#### NearMiss

In [ ]:
# ros = NearMiss(version=1)
# x_train_samp, y_train_samp = ros.fit_resample(x_train, y_train)
# print(sorted(Counter(y_train_samp).items()))
# modelling_mulitple(StandardScaler().fit_transform(x_train_samp), y_train_samp)

## Combined Over & UnderSampling

##### SMOTETomek

In [ ]:
# ros = SMOTETomek(random_state = random_state)
# x_train_samp, y_train_samp = ros.fit_resample(x_train, y_train)
# print(sorted(Counter(y_train_samp).items()))
# modelling_mulitple(StandardScaler().fit_transform(x_train_samp), y_train_samp)

##### SMOTEENN

In [ ]:
# ros = SMOTEENN(random_state = random_state)
# x_train_samp, y_train_samp = ros.fit_resample(x_train, y_train)
# print(sorted(Counter(y_train_samp).items()))
# modelling_mulitple(StandardScaler().fit_transform(x_train_samp), y_train_samp)

## FEATURE SELECTION

##### Feed Forward (Support Vector Machine) (F1 Score)

In [ ]:
y_train.value_counts()

In [ ]:
def classifier(x_train, y_train,scoring):
    
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)
    classifiers = LogisticRegression()
    scores = cross_val_score(classifiers, MinMaxScaler().fit_transform(x_train), y_train, scoring=scoring, cv=cv)
    
    return scores.mean()

In [ ]:
def feed_forward(unprocessed_cols,current_bestcols,scoring):
    
    results = [classifier(StandardScaler().fit_transform(x_train[current_bestcols + [x]]), y_train,scoring) for x in unprocessed_cols]
    current_bestcols = current_bestcols + [unprocessed_cols[results.index(max(results))]]
    bestcols.append((max(results), current_bestcols))
    unprocessed_cols.pop(results.index(max(results)))
    if len(unprocessed_cols) > 0:
        print(f"{len(unprocessed_cols)} columns left to process")
        feed_forward(unprocessed_cols,current_bestcols,scoring)
    else:
        return bestcols

In [ ]:
plt.rcParams.update(plt.rcParamsDefault)

In [ ]:
bestcols = []
cols = x_train.columns.tolist()
feed_forward(cols,bestcols,scoring='f1_macro')
sns.lineplot(x=[len(x[1]) for x in bestcols], y=[x[0] for x in bestcols], palette='Set2')
plt.xlabel('Number of Columns')
plt.ylabel('Scores')
plt.show()

In [ ]:
bestcols

In [ ]:
# bestcols = []
# cols = x_train.columns.tolist()
# feed_forward(cols,bestcols,scoring=tss_scorer)
# sns.lineplot(x=[len(x[1]) for x in bestcols], y=[x[0] for x in bestcols], palette='Set2')
# plt.xlabel('Number of Columns')
# plt.ylabel('Scores')
# plt.show()

In [ ]:
# bestcols = []
# cols = x_train.columns.tolist()
# feed_forward(cols,bestcols,scoring=hss2_scorer)
# sns.lineplot(x=[len(x[1]) for x in bestcols], y=[x[0] for x in bestcols], palette='Set2')
# plt.xlabel('Number of Columns')
# plt.ylabel('Scores')
# plt.show()

## NOISE REDUCTION

##### DBSCAN

In [ ]:
# # Use random Undersampling to stabilize the data
# ros = RandomUnderSampler(random_state = random_state)
# x_train_samp, y_train_samp = ros.fit_resample(x_train, y_train)
# print(sorted(Counter(y_train_samp).items()))
# modelling_mulitple(MinMaxScaler().fit_transform(x_train_samp), y_train_samp)

In [ ]:
# Sampled data
data = StandardScaler().fit_transform(x_train_samp[bestcols[5][1]])

In [ ]:
# Optimal Eps
#https://medium.com/@tarammullin/dbscan-parameter-estimation-ff8330e3a3bd
neighbors = NearestNeighbors(n_neighbors=5)
neighbors_fit = neighbors.fit(data)
distances, indices = neighbors_fit.kneighbors(data)

distances = np.sort(distances, axis=0)
distances = distances[:,1]
plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams["figure.figsize"] = [7.00, 5.00]

plt.plot(distances)
plt.show()

In [ ]:
db = DBSCAN(eps = 0.25, min_samples = 5).fit(data) 
labels = db.labels_

In [ ]:
# axes instance
fig = plt.figure(figsize=(6,6))
ax = Axes3D(fig, auto_add_to_figure=False)
fig.add_axes(ax)

# get colormap from seaborn
cmap = ListedColormap(sns.color_palette("husl", 256).as_hex())

# plot
sc = ax.scatter([x[0] for x in data], [x[1] for x in data], [x[2] for x in data], s=40, c=labels, marker='o', alpha=1)
# sc = ax.scatter(data[data.columns[0]], data[data.columns[1]], data[data.columns[2]], s=40, c=labels, marker='o', alpha=1)
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

# legend
plt.legend(*sc.legend_elements(), bbox_to_anchor=(1.05, 1), loc=2)

# save
plt.show()

In [ ]:
idxs = [idx for idx, value in enumerate(labels.tolist()) if value == -1]
to_check = x_train_samp.index.tolist()
noise = [value for idx,value in enumerate(to_check) if idx in idxs]

In [ ]:
# modelling_mulitple(x_train_samp[x_train_samp.index.isin(noise)], y_train_samp[y_train_samp.index.isin(noise)])

In [ ]:
# 0.917269	0.004286	0.834551	0.008577	0.834553	0.008574
modelling_mulitple(MinMaxScaler().fit_transform(x_train_samp[~x_train_samp.index.isin(noise)][bestcols[5][1]]), y_train_samp[~y_train_samp.index.isin(noise)])

In [ ]:
dt.head()

In [ ]:
target

In [ ]:
features

In [ ]:
y_train.shape